<a href="https://colab.research.google.com/github/rubuntu/Taller_Introduccion_a_Ciencia_de_Datos_IA_e_Ingenieria_de_Datos/blob/main/sesion_12_clasificacion_de_perros_y_gatos_con_resnet18_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación de Perros y Gatos con ResNet18 (Transfer Learning)

## Objetivos
- Aprender a cargar datasets de imágenes desde HuggingFace (`microsoft/cats_vs_dogs`).
- Preparar un pipeline de preprocesamiento con **transformaciones y data augmentation**.
- Usar un modelo preentrenado (**ResNet18 con pesos de ImageNet**) y adaptarlo al problema de clasificación binaria.
- Practicar el concepto de **congelar capas y entrenar solo las últimas** (fine-tuning parcial).
- Implementar **entrenamiento con early stopping** para evitar overfitting.
- Evaluar el modelo y visualizar ejemplos de predicciones correctas e incorrectas.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchvision.models import resnet18, ResNet18_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

## 1. Cargar dataset público (HuggingFace)

In [ ]:
dataset = load_dataset("microsoft/cats_vs_dogs")

# Dividir en 80% train / 20% test
dataset = dataset["train"].train_test_split(test_size=0.2)

print(dataset)

## 2. Transformaciones (Data Augmentation + Normalización)

In [ ]:
transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.Lambda(lambda img: img.convert("RGB")),  # ⚠️ convertir a RGB
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

transform_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.Lambda(lambda img: img.convert("RGB")),  # ⚠️ convertir a RGB
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

## 3. Adaptador HuggingFace → PyTorch Dataset

In [ ]:
class CatsDogsDataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.data = hf_dataset
        self.transform = transform
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        img = self.data[idx]["image"]
        label = self.data[idx]["labels"]   # ⚠️ usar 'labels' en lugar de 'file'

        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.long)

train_ds = CatsDogsDataset(dataset["train"], transform=transform_train)
test_ds  = CatsDogsDataset(dataset["test"], transform=transform_test)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
test_dl  = DataLoader(test_ds, batch_size=32)

## 4. Definir modelo (ResNet18 con pesos de ImageNet)

In [ ]:
weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)

# Congelar todas las capas excepto las últimas
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Reemplazar la última capa para 2 clases
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

## 5. Entrenamiento con Early Stopping

In [ ]:
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

#n_epochs = 20
n_epochs = 1
patience = 3
best_acc = 0
epochs_no_improve = 0

for epoch in range(n_epochs):
    # --- Entrenamiento ---
    model.train()
    running_loss = 0.0
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        preds = model(xb)
        loss = loss_fn(preds, yb)

        opt.zero_grad()
        loss.backward()
        opt.step()
        running_loss += loss.item()

    # --- Evaluación ---
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for xb, yb in test_dl:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb).argmax(dim=1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)

    acc = correct / total
    print(f"Epoch {epoch+1}, Loss={running_loss/len(train_dl):.4f}, Val Acc={acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        epochs_no_improve = 0
        torch.save(model.state_dict(), "best_catsdogs.pth")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping activado")
            break

print("Mejor accuracy alcanzado:", best_acc)

## 6. Visualización de predicciones

In [ ]:

images, labels = next(iter(test_dl))
images, labels = images.to(device), labels.to(device)
preds = model(images).argmax(dim=1)

plt.figure(figsize=(12,6))
for i in range(8):
    plt.subplot(2,4,i+1)
    img = images[i].cpu().permute(1,2,0).numpy()
    img = (img * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]).clip(0,1)  # desnormalizar
    plt.imshow(img)
    plt.title(f"Real: {labels[i].item()}, Pred: {preds[i].item()}")
    plt.axis("off")
plt.show()

## Preguntas de Discusión

1. ¿Qué ventajas ofrece usar un modelo preentrenado (transfer learning) frente a entrenar desde cero?
2. ¿Por qué es útil congelar capas en el inicio del entrenamiento y ajustar solo la última capa?
3. ¿Cómo ayuda el *data augmentation* a mejorar la capacidad de generalización del modelo?
4. ¿Qué diferencias observas en el rendimiento al descongelar más capas para el fine-tuning?
5. ¿Qué rol cumple la normalización con los valores de ImageNet en la estabilidad del entrenamiento?
6. ¿Cómo decide el early stopping cuándo detener el entrenamiento y por qué es importante?
7. ¿Qué métricas adicionales (además de accuracy) podrían ser útiles en este problema?
8. ¿Cómo se podría mejorar aún más el modelo si se dispusiera de más recursos computacionales?

## 💡 Preguntas de Discusión (desarrolladas)

1. **¿Qué ventajas ofrece usar un modelo preentrenado (transfer learning) frente a entrenar desde cero?**

   * Entrenar desde cero requiere grandes cantidades de datos y mucho tiempo de cómputo.
   * Los modelos preentrenados en ImageNet ya han aprendido características generales (bordes, texturas, formas), que son útiles para muchos problemas de visión.
   * Con *transfer learning*, solo se adapta la parte final de la red al nuevo conjunto de clases, logrando **mejor rendimiento con menos datos y menos tiempo de entrenamiento**.

---

2. **¿Por qué es útil congelar capas en el inicio del entrenamiento y ajustar solo la última capa?**

   * Las primeras capas de una CNN aprenden características muy generales (líneas, bordes, patrones de color).
   * Si se ajustan todas desde el inicio, se corre el riesgo de *desaprender* esas representaciones útiles.
   * Congelarlas permite entrenar más rápido y reducir el riesgo de sobreajuste, enfocando el aprendizaje solo en la capa de clasificación final.

---

3. **¿Cómo ayuda el *data augmentation* a mejorar la capacidad de generalización del modelo?**

   * Genera versiones modificadas de las imágenes (rotadas, espejadas, con variaciones de color).
   * Esto obliga al modelo a aprender **patrones invariantes** a pequeñas transformaciones, en lugar de memorizar ejemplos concretos.
   * Mejora la robustez y reduce el riesgo de sobreajuste cuando los datasets son pequeños.

---

4. **¿Qué diferencias observas en el rendimiento al descongelar más capas para el fine-tuning?**

   * Congelar casi todo → entrenamiento rápido pero menos capacidad de adaptación al nuevo dominio.
   * Descongelar últimas capas → mejor ajuste al dataset objetivo, a costa de más tiempo de entrenamiento.
   * Descongelar toda la red → mayor capacidad de adaptación, pero mayor riesgo de sobreajuste si el dataset es pequeño.
   * En la práctica, **descongelar gradualmente** (empezando desde las últimas capas) suele dar los mejores resultados.

---

5. **¿Qué rol cumple la normalización con los valores de ImageNet en la estabilidad del entrenamiento?**

   * Los modelos preentrenados esperan entradas con la misma estadística que los datos de ImageNet.
   * Normalizar con `mean=[0.485, 0.456, 0.406]` y `std=[0.229, 0.224, 0.225]` alinea la distribución de píxeles con la que el modelo fue entrenado originalmente.
   * Esto evita desajustes que podrían degradar el rendimiento o dificultar la convergencia.

---

6. **¿Cómo decide el early stopping cuándo detener el entrenamiento y por qué es importante?**

   * Early stopping monitorea una métrica de validación (ej. pérdida o accuracy).
   * Si no mejora después de un número definido de épocas (*patience*), se detiene el entrenamiento.
   * Esto evita que el modelo siga ajustándose al conjunto de entrenamiento mientras empeora en el de validación (sobreajuste).
   * También ahorra tiempo y recursos.

---

7. **¿Qué métricas adicionales (además de accuracy) podrían ser útiles en este problema?**

   * **Precisión (precision):** proporción de predicciones positivas correctas (útil si queremos pocas falsas alarmas).
   * **Recall (sensibilidad):** proporción de verdaderos positivos detectados (útil si no queremos dejar escapar casos).
   * **F1-score:** balance entre precisión y recall.
   * **Matriz de confusión:** para ver qué clases se confunden más.
   * **AUC-ROC:** mide la capacidad de distinguir entre clases en distintos umbrales.

---

8. **¿Cómo se podría mejorar aún más el modelo si se dispusiera de más recursos computacionales?**

   * Usar arquitecturas más grandes y potentes (**ResNet50, EfficientNet, Vision Transformers**).
   * Entrenar durante más épocas con estrategias de regularización (dropout, weight decay).
   * Aumentar la resolución de entrada (224×224 → 384×384).
   * Usar *ensembles* de varios modelos para combinar predicciones.
   * Aplicar *semi-supervised learning* o *self-supervised pretraining* para aprovechar datos no etiquetados.

